In [1]:
import os
import csv
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

plt.rcParams['figure.figsize'] = [10, 5]

%matplotlib notebook

In [2]:
path = 'z:\warfarin'

In [3]:
global df, filename

df = None
filename = ''

@interact(penalty_func=['stdev', 'change_count'],
          penalty_coef=[0.25, 0.50, 0.75, 1, 2, 5, 10],
          model=['training', 'test', 'both'],
          stat=['TTR', 'dose_change', 'delta_dose'],
          normal=True,
          sensitive=False,
          highly_sensitive=False)
def plot_file(penalty_func, penalty_coef, model, stat, normal, sensitive, highly_sensitive):
    global df, filename
    
    plt.close()

    filename_temp = f'W90dayd_10INR_10Td_chg_coef_{penalty_coef:2.2f}func{penalty_func}ph1_90ph1chg_15mxd1_15ph2d_1ph2chg_15DQNbwd(32,32,32)g0.95efnbff900clrFbtch50vld0.3.txt'

    chosen_model = {'training': ["('protocol', 'training')"],
                   'test': ["('protocol', 'test')"],
                   'both': ["('protocol', 'training')", "('protocol', 'test')"]}[model]
    sensitivity = (['normal'] if normal else []) + (['sensitive'] if sensitive else []) + (['highly sensitive'] if highly_sensitive else [])
    
    if df is None or filename != filename_temp:
        filename = filename_temp
        df = pd.read_csv(os.path.join(path, filename), delimiter='\t', names=['run', 'model', 'stat', 'sensitivity', 'value'])

    table = pd.pivot_table(df[df.stat == stat], values='value', index=['run'],
                           columns=['model', 'sensitivity'], aggfunc=np.mean)
    data = table[[(m, s) for m in chosen_model for s in sensitivity if (m, s) != ("('protocol', 'test')", 'highly sensitive')]]
    plt.figure(figsize = (16, 6))
    plt.plot(data)
    plt.legend(data.columns)

interactive(children=(Dropdown(description='penalty_func', options=('stdev', 'change_count'), value='stdev'), …

In [4]:
file_indices = {}
for name in glob.iglob(os.path.join(path, 'W90dayd_10INR_10Td_chg_coef_*func*ph1_90ph1chg_15mxd1_15ph2d_1ph2chg_15DQNbwd(32,32,32)g0.95efnbff900clrFbtch50vld0.3*.csv')):
    try:
        fname = name[name.rfind('\\')+1:-8]
        file_indices[fname] = max(file_indices[fname], int(name[-8:-4]))
    except KeyError:
        file_indices[fname] = int(name[-8:-4])

In [5]:
global df_sp, filename_sp

df_sp = None
filename_sp = ''

@interact(penalty_func=['stdev', 'change_count'],
          penalty_coef=[0.25, 0.50, 0.75, 1, 2, 5, 10],
          model=['training', 'test'])
def sample_path(penalty_func, penalty_coef, model):
    global df_sp, filename_sp

    plt.close()

    name = f'W90dayd_10INR_10Td_chg_coef_{penalty_coef:2.2f}func{penalty_func}ph1_90ph1chg_15mxd1_15ph2d_1ph2chg_15DQNbwd(32,32,32)g0.95efnbff900clrFbtch50vld0.3'
    num = f'{file_indices[name]:04}'

    filename_temp = name + num + '.csv'

    if df_sp is None or filename_sp != filename_temp:
        filename_sp = filename_temp
        df_sp = pd.read_csv(os.path.join(path, filename_sp), index_col=0)

    df_sp_slice = df_sp[df_sp['agent/subject'] == f"('protocol', \'{model}\')"]
    patient = list(df_sp_slice[['age', 'CYP2C9', 'VKORC1']].iloc[0])
    data = df_sp_slice[df_sp_slice.instance_id == df_sp_slice.instance_id.max()][['action', 'reward', 'INR']]

    fig, axs = plt.subplots(3, figsize = (16, 6))
    fig.suptitle(str(patient))
    axs[0].bar(data.index, data['action'])
    axs[0].set_title('action')
    axs[1].plot(data['INR'])
    axs[1].plot(list(data.index), [2]*len(data.index), list(data.index), [3]*len(data.index))
    axs[1].set_title('INR')
    axs[2].plot(data['reward'])
    axs[2].set_title('penalty')

interactive(children=(Dropdown(description='penalty_func', options=('stdev', 'change_count'), value='stdev'), …